
# Column Transformer with Mixed Types


This example illustrates how to apply different preprocessing and feature
extraction pipelines to different subsets of features, using
`ColumnTransformer`. This is particularly handy for the
case of datasets that contain heterogeneous data types, since we may want to
scale the numeric features and one-hot encode the categorical ones.

In this example, the numeric data is standard-scaled after mean-imputation,
while the categorical data is one-hot encoded after imputing missing values
with a new category (``'missing'``).

In addition, we show two different ways to dispatch the columns to the
particular pre-processor: by column names and by column data types.

Finally, the preprocessing pipeline is integrated in a full prediction pipeline
using `Pipeline`, together with a simple classification
model.


In [1]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV


#### Step 1: Read the data set `Titanic.csv`
Print the column names. Drop 'boat','body','cabin','home.dest','name','ticket','sibsp','parch'

In [3]:
df = pd.read_csv('Titanic.csv')
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"


In [55]:
df2 = df.drop(['boat','body','cabin','home.dest','name','ticket','sibsp','parch'],axis=1)

In [6]:
df2.columns

Index(['pclass', 'survived', 'sex', 'age', 'fare', 'embarked'], dtype='object')

#### Step 2: Calculate duplicates
- Report the size of the dataset
- Report if there are any duplicates using `duplicated()` method of pandas dataframe
- List all duplicate rows
- Drop all duplicate rows using `dop_duplicates` method of pandas dataframe
- Report the new shape of the dataset

In [7]:
df2.shape

(1309, 6)

In [8]:
df2.duplicated()

,0
0,False
1,False
2,False
3,False
4,False
...,...
1304,False
1305,True
1306,False
1307,False


In [10]:
df2.drop_duplicates(inplace = True)

In [11]:
df2.shape

(1094, 6)

#### Step 3: Explore the dataset for missing values
- Assign `survived' as target variable `y` and the rest as `X`
- Print info about `X` to see if there are any null values and the type of the features
- Check if all columns have only alpha numeric characters. To do that you need to use `isalnum()` method of Python which works on strings. Before using this method. we need to change all features into string using `astype('str')`
- Replace all `?` characters with `np.nan`
- Now, turn 'age','fare' into numeric again using `apply(pd.to_numeric) `
- Print the first five samples of `X`
- Summarize the number of unique values in each column using `nunique()`

In [12]:
X = df2.drop('survived',axis=1)
y = df2['survived']

In [13]:
X.info() #(dot) and any other alpha numeric character be take care of them

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pclass    1094 non-null   int64 
 1   sex       1094 non-null   object
 2   age       1094 non-null   object
 3   fare      1094 non-null   object
 4   embarked  1094 non-null   object
dtypes: int64(1), object(4)
memory usage: 51.3+ KB


In [14]:
cols = X.columns
X = X[cols].astype(str)

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pclass    1094 non-null   object
 1   sex       1094 non-null   object
 2   age       1094 non-null   object
 3   fare      1094 non-null   object
 4   embarked  1094 non-null   object
dtypes: object(5)
memory usage: 51.3+ KB


In [17]:
X.isnull()

,pclass,sex,age,fare,embarked
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
1301,False,False,False,False,False
1304,False,False,False,False,False
1306,False,False,False,False,False
1307,False,False,False,False,False


In [18]:
X.isnull().sum()

,0
pclass,0
sex,0
age,0
fare,0
embarked,0


In [19]:
X.head()

,pclass,sex,age,fare,embarked
0,1,female,29,211.3375,S
1,1,male,0.9167,151.55,S
2,1,female,2,151.55,S
3,1,male,30,151.55,S
4,1,female,25,151.55,S


In [20]:
X.loc[3, 'age'] # index

'30'

In [21]:
# order no
X.iloc[3,2] # 2 is order of age column

'30'

In [24]:
X.iloc[:,2]

,age
0,29
1,0.9167
2,2
3,30
4,25
...,...
1301,45.5
1304,14.5
1306,26.5
1307,27


In [25]:
X.iloc[:,2].str.isalnum()

,age
0,True
1,False
2,True
3,True
4,True
...,...
1301,False
1304,False
1306,False
1307,True


In [26]:
(~X.iloc[:,2].str.isalnum()).sum()

172

In [27]:
X.iloc[:,2].str.isalnum()

,age
0,True
1,False
2,True
3,True
4,True
...,...
1301,False
1304,False
1306,False
1307,True


In [28]:
~X.iloc[:,2].str.isalnum()==True

,age
0,False
1,True
2,False
3,False
4,False
...,...
1301,True
1304,True
1306,True
1307,False


In [29]:
X[~X.iloc[:,2].str.isalnum()==True]

,pclass,sex,age,fare,embarked
1,1,male,0.9167,151.55,S
15,1,male,?,25.925,S
37,1,male,?,26.55,S
40,1,male,?,39.6,C
46,1,male,?,31,S
...,...,...,...,...,...
1291,3,male,?,8.7125,S
1294,3,male,28.5,16.1,S
1301,3,male,45.5,7.225,C
1304,3,female,14.5,14.4542,C


In [31]:
X = X.replace('?',np.nan)

In [34]:
X.iloc[:,2].str.isalnum()[:20]

,age
0,True
1,False
2,True
3,True
4,True
5,True
6,True
7,True
8,True
9,True


In [35]:
type(np.nan)

float

In [36]:
X[['age', 'fare']].dtypes

,0
age,object
fare,object


In [37]:
X[['age', 'fare']].apply(pd.to_numeric)

,age,fare
0,29.0000,211.3375
1,0.9167,151.5500
2,2.0000,151.5500
3,30.0000,151.5500
4,25.0000,151.5500
...,...,...
1301,45.5000,7.2250
1304,14.5000,14.4542
1306,26.5000,7.2250
1307,27.0000,7.2250


In [39]:
ty1 = {'age':float, 'fare':float}
X = X.astype(ty1)

In [40]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1094 non-null   object 
 1   sex       1094 non-null   object 
 2   age       966 non-null    float64
 3   fare      1093 non-null   float64
 4   embarked  1092 non-null   object 
dtypes: float64(2), object(3)
memory usage: 83.6+ KB


In [41]:
X['embarked'] = X['embarked'].astype(str)

In [42]:
np.unique(X['embarked'])

array(['C', 'Q', 'S', 'nan'], dtype=object)

#### Step 4: Use `ColumnTransformer` by selecting column by names

##############################################################################

 We will train our classifier with the following features:

 Numeric Features:

 * ``age``: float;
 * ``fare``: float.

 Categorical Features:

 * ``embarked``: categories encoded as strings ``{'C', 'S', 'Q'}``;
 * ``sex``: categories encoded as strings ``{'female', 'male'}``;
 * ``pclass``: ordinal integers ``{1, 2, 3}``.

 Create the preprocessing pipelines for both numeric and categorical data.
 - For numeric features, create a pipeline with using imputer with median strategy and standard scaler
 - For categorical features, use imputer with most frequent and onehot encoder

 Use columntransformer to transform the features

 Append the preprocessor to the KNeighborsClassifier


In [60]:
numeric_features = ['age', 'fare']
categorical_features = ['embarked', 'sex', 'pclass']

In [61]:
numeric_transformer = Pipeline(steps=[
    ('numimp', SimpleImputer(strategy='median')),\
    ('scaler', StandardScaler())])

In [62]:
categorical_transformer = Pipeline(steps=[
    ('catimp', SimpleImputer(strategy='most_frequent', missing_values = np.NaN)),\
    ('ohe', OneHotEncoder(handle_unknown='ignore'))])

In [63]:
tr =ColumnTransformer(transformers= [('num', numeric_transformer, numeric_features),\
('cat', categorical_transformer, categorical_features)])

In [64]:
tr

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('numimp',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['age', 'fare']),
                                ('cat',
                                 Pipeline(steps=[('catimp',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['embarked', 'sex', 'pclass'])])

In [65]:
clf = Pipeline(steps=[('prep', tr),\
                      ('Knn', KNeighborsClassifier())])

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y,random_state = 1)

In [67]:
clf.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('numimp',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('catimp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['embarked', 'sex',
                                                   'pclass'])])),
                ('Knn', KNeighborsClassifier())])

In [69]:
clf.score(X_test, y_test)

0.7737226277372263

Using the prediction pipeline in a grid search

##############################################################################

 Grid search can also be performed on the different preprocessing steps
 defined in the `ColumnTransformer` object, together with the classifier's
 hyperparameters as part of the `Pipeline`.
 We will search for both the imputer strategy of the numeric preprocessing
 and the number of neighbor parameter of the kneighbors classifier using
 `GridSearchCV`.



In [72]:
params= {
    'prep__num__numimp__strategy': ['mean', 'median'],
    'Knn__n_neighbors': np.arange(1,30,2)}

In [73]:
mygrid = GridSearchCV(clf, param_grid=params, cv=5)

In [74]:
mygrid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('numimp',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('catimp',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['embarked',
                                                                          'sex',
                                                                          'pclass'])])),
                                       ('Knn', KNeighborsClassifier())]),
             param_grid={'Knn__n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29]),
                         'prep__num__numimp__strategy': ['mean', 'median']})

In [75]:
mygrid.best_params_

{'Knn__n_neighbors': 9, 'prep__num__numimp__strategy': 'mean'}